In [2]:
from langchain_community.agent_toolkits import FileManagementToolkit

toolkit = FileManagementToolkit()

In [3]:
toolkit.get_tools()

[CopyFileTool(),
 DeleteFileTool(),
 FileSearchTool(),
 MoveFileTool(),
 ReadFileTool(),
 WriteFileTool(),
 ListDirectoryTool()]

In [4]:
list_dir = toolkit.get_tools()[6]

In [5]:
result = list_dir.invoke(input= "C:\\KshitijDocs")

In [7]:
result

'4thSemResult.pdf\nAadhaarcard.pdf\nAadharCard.pdf\nacademic credits.pdf\nAnnotation 2024-07-15 103126.png\nbittuexamfee.pdf\nchotudocs\ndownload (1).pdf\ndownload.pdf\nDSAwithC++.pdf\nExam form Fees 5th sem.pdf\nexamform4thsem.pdf\nin.gov.cbse-SSCER-191677312023.pdf\nkshitij5thsem1stInstallment.pdf\nKshitijDocs\nKshitijResume.pdf\nOffer_Letter _ Webmobi360 - AIML Engineer_ Kshitij kumrawat.pdf\npassport_pic.jpg\nPicsart_24-05-21_11-40-59-477.jpg\nprofile.jpg\nprofile_pic.jpg\nRECORD OF CERTIFICATION - 1 (1).pdf\nRECORD OF CERTIFICATION.xlsx\nScreenshot 2024-12-18 181902.png\nshaily5thsem1stinstallment.pdf\nUigGeeksCertificate.png'

In [9]:
type(result.split("\n"))

list

In [41]:
import requests
url = f"http://127.0.0.1:8888/?s={"pancard"}&json=1&path=1&path_column=1"  # add json=1 to get JSON response

url

'http://127.0.0.1:8888/?s=pancard&json=1&path=1&path_column=1'

In [62]:
## Everything tool 

import requests

def search_file(query:str)->str:
    url = f"http://127.0.0.1:8888/?s={query}&json=1&path_column=1"  # add json=1 to get JSON response
    resp = requests.get(url)
    # print(resp)
    resp.raise_for_status()
    file_paths = resp.json()
    print(file_paths)  # parse JSON response
    search_results = ''
    for file in file_paths['results']: 
        search_results += f"{file['path']+'\\'+file['name']} ;"
    print(search_results)
    return search_results

# Example usage
file_paths = search_file("pancard")
# print("Found file paths:")
# for path in file_paths:
#     print(path)


{'totalResults': 1, 'results': [{'type': 'file', 'name': 'pancard.jpg', 'path': 'C:\\KshitijDocs\\KshitijDocs'}]}
C:\KshitijDocs\KshitijDocs\pancard.jpg ;


In [51]:
search_results = ''
for file in file_paths['results']: 
    search_results += f"{file['name']} ;"
print(search_results)

1812cd35-d43f-4f4e-802a-d91f9b66d6f7_KshitijKumrawat (7).pdf ;2c8c8d1a-edbd-4bb4-a193-0d7912e2ef60_KshitijKumrawat (7).pdf ;32cc7b3e-5bf3-488e-a8a9-8dd2a18d62b5_KshitijKumrawat (8).pdf ;32cc7b3e-5bf3-488e-a8a9-8dd2a18d62b5_KshitijKumrawat (8).pdf ;3bb2a11b-5b0d-4e1f-ab6e-c66cd805b6dd_KshitijKumrawat (7).pdf ;3bb2a11b-5b0d-4e1f-ab6e-c66cd805b6dd_KshitijKumrawat (7).pdf ;9a9548db-afa0-42a2-bc42-57a544d2ca7f_KshitijKumrawat (8).pdf ;9bf0e4b9-bc97-4472-9d19-6b6e07af1117_KshitijKumrawat (10).pdf ;9f09e2c1-21bf-4578-962a-5e42fd4ed40b_KshitijKumrawat (8).pdf ;ba9fbb14-c2c2-43e5-afa2-91e4325dbd0b_KshitijKumrawat (10).pdf ;ba9fbb14-c2c2-43e5-afa2-91e4325dbd0b_KshitijKumrawat (10).pdf ;KshitijKumrawat (1).pdf ;KshitijKumrawat (10).lnk ;KshitijKumrawat (10).pdf ;KshitijKumrawat (2).pdf ;KshitijKumrawat (3).pdf ;KshitijKumrawat (4) (1).pdf ;KshitijKumrawat (4).pdf ;KshitijKumrawat (5).pdf ;KshitijKumrawat (6) (1).pdf ;KshitijKumrawat (6).pdf ;KshitijKumrawat (7).pdf ;KshitijKumrawat (8).pdf ;Kshit

In [26]:
import subprocess

def everything_cli_search(query):
    # Path to Everything CLI executable (adjust if needed)
    es_path = r"C:\Program Files\Everything\Everything.exe"
    args = [es_path, "-n", query]  # -n for newline separated results
    
    try:
        result = subprocess.run(args, capture_output=True, text=True, check=True)
        # Split the output by newline to get file paths
        file_paths = result.stdout.strip().split('\n')
        return file_paths
    except subprocess.CalledProcessError as e:
        print(f"Error executing Everything CLI: {e}")
        return []

# Usage example
files = everything_cli_search("pancard")
print("Found files:")
for f in files:
    print(f)


Found files:



In [14]:
import os

def list_all_files(root_path, extensions=None):
    """
    Recursively list all files under root_path that match given extensions.

    :param root_path: Directory to scan
    :param extensions: Iterable of allowed extensions (e.g. {".pdf", ".jpg"})
    :return: List of file paths
    """
    if extensions is not None:
        extensions = {ext.lower() for ext in extensions}

    all_files = []
    for dirpath, dirnames, filenames in os.walk(root_path):
        for fname in filenames:
            if extensions:
                if not os.path.splitext(fname)[1].lower() in extensions:
                    continue
            all_files.append(os.path.join(dirpath, fname))
    return all_files


# Example usage: only PDFs and common images
all_files = list_all_files(
    "C:\\",
    extensions={".pdf"}
)

print(f"Found {len(all_files)} files")


Found 616 files


In [12]:
import os

def list_all_files(root_path):
    all_files = []
    for dirpath, dirnames, filenames in os.walk(root_path):
        for fname in filenames:
            all_files.append(os.path.join(dirpath, fname))
    return all_files

all_files = list_all_files("C:\\")


In [13]:
all_files

['C:\\DumpStack.log',
 'C:\\DumpStack.log.tmp',
 'C:\\hiberfil.sys',
 'C:\\logUploaderSettings.ini',
 'C:\\logUploaderSettings_temp.ini',
 'C:\\pagefile.sys',
 'C:\\swapfile.sys',
 'C:\\UC-b2b06b34-0a21-4d2d-923e-57a5f9032c8e.pdf',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I006NJ6.pdf',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I00O3JQ.pdf',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I0C7693.mkv',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I0FYDAR.pdf',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I137VLC.pdf',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I1P4B2S',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I23O6LE.venv',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I29BTMR.pdf',
 'C:\\$Recycle.Bin\\S-1-5-21-1971200603-2044776189-1541449858-1001\\$I2IA86X.pdf',
 'C:\\$Recycle.Bin\\S-1-

In [ ]:
%pwd

In [ ]:
import os 
os.startfile("E:\Generative_AI_Roadmap.pdf")

In [ ]:
toolkit = FileManagementToolkit(
    root_dir=str("'c:\\code\\local_agent'")
)  # If you don't provide a root_dir, operations will default to the current working directory
toolkit.get_tools()

In [ ]:
tools = FileManagementToolkit(
        selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
tools

In [1]:
import os
import numpy as np 
# from 
# def calculate_similarity(tex1,)
def list_specific_files(directory_path):
    """
    Lists only PDF, PPT, and Excel, images  files in the specified directory with with all the subdirectory files.

    Args:
        directory_path (str): The path to the directory to scan.

    Returns:
        list: A list of full file paths (strings) matching the criteria.
    """
    allowed_extensions = ('.pdf', '.ppt', '.pptx', '.xls', '.xlsx','.csv', '.doc', '.docx', '.jpg', '.jpeg', '.png')
    
    filtered_files = []
    
    # Walk through directory and all subdirectories
    for root, dirs, files in os.walk(directory_path):
        for i,file in enumerate(files):
            if file.lower().endswith(allowed_extensions):
                if i==0:
                    filtered_files.append(os.path.join(root, file))
                else:
                    filtered_files.append(os.path.join(file))
    
    return filtered_files

# Example usage:
directory = 'C:\\KshitijDocs' # Use current directory for demonstration
# directory = '/path/to/your/directory' # Replace with your target directory

specific_files = list_specific_files(directory)
print(specific_files)

['C:\\KshitijDocs\\4thSemResult.pdf', 'Aadhaarcard.pdf', 'AadharCard.pdf', 'academic credits.pdf', 'Annotation 2024-07-15 103126.png', 'bittuexamfee.pdf', 'download (1).pdf', 'download.pdf', 'DSAwithC++.pdf', 'Exam form Fees 5th sem.pdf', 'examform4thsem.pdf', 'in.gov.cbse-SSCER-191677312023.pdf', 'kshitij5thsem1stInstallment.pdf', 'KshitijResume.pdf', 'Offer_Letter _ Webmobi360 - AIML Engineer_ Kshitij kumrawat.pdf', 'passport_pic.jpg', 'Picsart_24-05-21_11-40-59-477.jpg', 'profile.jpg', 'profile_pic.jpg', 'RECORD OF CERTIFICATION - 1 (1).pdf', 'RECORD OF CERTIFICATION.xlsx', 'Screenshot 2024-12-18 181902.png', 'shaily5thsem1stinstallment.pdf', 'UigGeeksCertificate.png', 'C:\\KshitijDocs\\chotudocs\\allotment letter.pdf', 'choice filling reciept.pdf', 'CLASSXCERTIFICATE.pdf', 'ConfirmationPage-250310468258.pdf', 'ConfirmationPage-session1.pdf', 'divyanshi aplication no.png', 'IMG-20250630-WA0001[1].jpg', 'IMG_20241110_215939633~2.jpg', 'IMG_20241110_220415746~2.jpg', 'PHOTO.jpg', 'reg

In [2]:
specific_files

['C:\\KshitijDocs\\4thSemResult.pdf',
 'Aadhaarcard.pdf',
 'AadharCard.pdf',
 'academic credits.pdf',
 'Annotation 2024-07-15 103126.png',
 'bittuexamfee.pdf',
 'download (1).pdf',
 'download.pdf',
 'DSAwithC++.pdf',
 'Exam form Fees 5th sem.pdf',
 'examform4thsem.pdf',
 'in.gov.cbse-SSCER-191677312023.pdf',
 'kshitij5thsem1stInstallment.pdf',
 'KshitijResume.pdf',
 'Offer_Letter _ Webmobi360 - AIML Engineer_ Kshitij kumrawat.pdf',
 'passport_pic.jpg',
 'Picsart_24-05-21_11-40-59-477.jpg',
 'profile.jpg',
 'profile_pic.jpg',
 'RECORD OF CERTIFICATION - 1 (1).pdf',
 'RECORD OF CERTIFICATION.xlsx',
 'Screenshot 2024-12-18 181902.png',
 'shaily5thsem1stinstallment.pdf',
 'UigGeeksCertificate.png',
 'C:\\KshitijDocs\\chotudocs\\allotment letter.pdf',
 'choice filling reciept.pdf',
 'CLASSXCERTIFICATE.pdf',
 'ConfirmationPage-250310468258.pdf',
 'ConfirmationPage-session1.pdf',
 'divyanshi aplication no.png',
 'IMG-20250630-WA0001[1].jpg',
 'IMG_20241110_215939633~2.jpg',
 'IMG_20241110_220

In [ ]:
read_tool, write_tool, list_tool = tools
# write_tool.invoke({"file_path": "example.txt", "text": "Hello World!"})

In [ ]:
# List files in the working directory
list_tool.invoke(input={"dir_path": "E:\\"})

In [ ]:
## shell Tool 
from langchain_community.tools import ShellTool

shell_tool = ShellTool(verbose=True, working_dir="C:\\code")
print(shell_tool.run({"commands": ["cd C:/code", "time"]}))

In [ ]:
import psutil 
result = ""
for disk in psutil.disk_partitions():
    result += disk.device + ";"
print(result)


In [ ]:
import psutil
    drives = []
    for partition in psutil.disk_partitions():
        try:
            partition_usage = psutil.disk_usage(partition.mountpoint)
            drives.append(f"{partition.device} ({partition.fstype}) - "
                                f"{partition_usage.free // (1024**3)} GB free")
        except PermissionError:
            drives.append(f"{partition.device} (Access Denied)")
            
    result = "Available drives:\n" + "\n".join(drives)
    print(f"[VERBOSE] Listed drives: {len(drives)} found")


In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode
from langchain.schema import AIMessage
from langgraph.checkpoint.memory import MemorySaver
import psutil

from typing import TypedDict
from typing_extensions import Literal
from langchain.schema import BaseMessage
from langchain.schema import messages_to_dict
from langchain.schema import messages_from_dict
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from langchain.schema import messages_to_dict
from langchain.schema import messages_from_dict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os
# 1. Define the state schema for the graph (messages for chat history)
state = MessagesState
from typing import Annotated
from langgraph.prebuilt import  tools_condition
from langchain_community.tools import ShellTool
from langchain_core.prompts import ChatPromptTemplate
shell_tool = ShellTool()

from langchain_community.agent_toolkits import FileManagementToolkit
# 2. Initialize the FileManagementToolkit with your root directory
root_dir = "C:\\"
toolkit = FileManagementToolkit()
FileManagementTools = toolkit.get_tools()

# 3. Initialize the LLM (e.g., Anthropic Claude)
# llm = init_chat_model("anthropic:claude-3-7-sonnet-latest")
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="qwen/qwen3-32b", api_key=api_key)
from langgraph.types import Command, interrupt

# 4. Define the node that calls the LLM with tools bound
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

from langchain_core.tools import tool

@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]

tools = [shell_tool, FileManagementTools[6], human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    
    return {"messages": [message]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)
result = ""
for disk in psutil.disk_partitions():
    result += disk.device + ";"

user_input = """""You are a helpful assistant designed to search files and folders on a Windows system.
        
        Your capabilities:
        - You have access to the following drives: {result}. *Must* ask the user which drive to search.
        - Use FileManagement tools to list directory contents
        - Use Shell tools to navigate and open files
        - Reason about file locations based on common Windows directory structures
        - Answer questions and provide guidance about file management
        
        Guidelines:
        - *Always* start by listing available drives when helping with file searches
        - Do not make any search command without first listing drives
        - Use tools to explore the file system step-by-step
        - Think through your approach before taking action
        - If you need more information from the user, ask directly in conversation
        - When you can't find something, explain what you searched and ask for clarification
        - Open files using their full Windows path format
        - Be thorough but efficient in your search strategy
        - Provide helpful suggestions when files aren't found
        
        Find hackathon solution ppt file and open it.""" 
config = {"configurable": {"thread_id": "10"}}

events = graph.stream(
    {"messages": user_input},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
user_input = """You are a helpful assistant designed to search the files and folders required in a Windows system. "
        "You have to do reasoning about where you can find the particular file then take decision. "
        "Use the FileManagement tool to list files in the directory, and if you find any folder related to that file then only provide the path. "
        "You have a shell tool so open the file with shell tool, don't ask, just open it. "
        "You can also take assistance from human by asking about where to find the file or folder if they have something in their memory. "
        "Don't be shy to ask or validate your searching because there can be thousands of folders or files. "
        "But make sure that when you ask human, you should provide them with some options to choose from."
        "But make make sure that if you can identify the file or folder yourself then don't ask human."
        "To open a file you have the open file tool so give the full path of the file in Windows format.
        
        Find my profile image""" 
config = {"configurable": {"thread_id": "11"}}

events = graph.stream(
    {"messages": user_input},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
user_input = "Give me some folder options ?"

In [ ]:
events = graph.stream(
    {"messages": user_input},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
user_input = "what my last question ?"
events = graph.stream(
    {"messages": user_input},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
human_response = (
    "not present here"

)
print("Human response:", human_response)
human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os 
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
# llm= ChatGroq(model="openai/gpt-oss-20b", api_key=api_key)
llm= ChatGroq(model="qwen/qwen3-32b", api_key=api_key)

In [ ]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

from langchain_community.agent_toolkits import FileManagementToolkit
# 2. Initialize the FileManagementToolkit with your root directory
root_dir = "C:\\"
toolkit = FileManagementToolkit(root_dir=root_dir)
FileManagementTools = toolkit.get_tools()

In [ ]:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.types import interrupt
import os

memory = MemorySaver()
# @tool
# def human_assistance_tool(query: str) -> str:
#     """Ask human for assistance. Assitance can be asking like if file you are asking in which which folder from the given list"""
#     human_response = interrupt({"query": query})
#     return human_response["data"]

@tool
def open_file(file_path: str) -> None:
    """Give the full path of the file you want to open according to windows format"""
    os.startfile(file_path)
    return "done"

# from langgraph.types import interrupt, Command
# @tool
# def human_feedback_node(state):
#     # Extract relevant information from the state to present to the human
#     message_to_human = state["messages"][-1].content if state["messages"] else "Please provide your input."
        
#     # Interrupt the graph and prompt for human input
#     human_input = interrupt({"message_for_human": message_to_human})
        
#     # Process the human_input and update the state as needed
#     # For example, you might add a HumanMessage to the messages list
#     # or update other state variables based on the feedback.
#     return {"human_feedback": human_input} # Example update

tools = [shell_tool,FileManagementTools[6], open_file]
agent = create_react_agent(
    llm,
    tools,
    prompt=(
        "You are a helpful assistant designed to search the files and folders required in a Windows system. "
        "You have to do reasoning about where you can find the particular file then take decision. "
        "Use the FileManagement tool to list files in the directory, and if you find any folder related to that file then only provide the path. "
        "You have a shell tool so open the file with shell tool, don't ask, just open it. "
        "You can also take assistance from human by asking about where to find the file or folder if they have something in their memory. "
        "Don't be shy to ask or validate your searching because there can be thousands of folders or files. "
        "To open a file you have the open file tool so give the full path of the file in Windows format."
    ),
    checkpointer=memory
)

config = {"configurable": {"thread_id": "6"}}

# events = graph.stream(
#     {"messages": user_input},
#     config,
#     stream_mode="values",
# )
# for event in events:
#     if "messages" in event:
#         event["messages"][-1].pretty_print()


input_message = {
    "role": "user",
    "content": (
        "Open my profile image "
    ),
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = {
    "role": "user",
    "content": (
        "Can you give me folder options ?"
    ),
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
## adding images encoding as tool
import base64

def encode_image(image_path):
    """Encode an image file to a Base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Example usage: Replace with your image file
image_path = "C:\\KshitijDocs\\KshitijDocs\\pancard.jpg"
base64_image = encode_image(image_path)

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# Initialize the OpenAI multimodal model
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1024)

# Create the message payload for the chat model
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Extract all the text from this image and list it clearly."
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
        },
    ]
)

# Invoke the model
response = llm.invoke([message])

# Print the extracted text
print(response.content)



In [2]:

## testing ollama model
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
llm = ChatOllama(
    model="granite4:latest",
    temperature=0,
    # other params...
)
llm.invoke([HumanMessage(content="Hello, Ollama!")])

AIMessage(content='Hello! How can I assist you today? If you have any questions or need help with something related to Ollama, feel free to ask.', additional_kwargs={}, response_metadata={'model': 'granite4:latest', 'created_at': '2025-11-08T07:09:03.6848675Z', 'done': True, 'done_reason': 'stop', 'total_duration': 25940084800, 'load_duration': 12059354400, 'prompt_eval_count': 14, 'prompt_eval_duration': 6824316000, 'eval_count': 31, 'eval_duration': 6996990600, 'model_name': 'granite4:latest'}, id='run--b7b12833-6c4d-4968-a649-fd990cceae2e-0', usage_metadata={'input_tokens': 14, 'output_tokens': 31, 'total_tokens': 45})

In [3]:
%pwd

'c:\\code\\local_agent\\Local_filesearch_agent'

In [2]:
import os, json, time

ALLOWED_EXTS = ('.pdf', '.doc', '.docx', '.jpg', '.jpeg', '.png')
EXCLUDE_DIRS = ["Windows", "Program Files", "Program Files (x86)", "PerfLogs", "$Recycle.Bin", "System Volume Information", "AppData", "Microsoft"]

def should_exclude(path):
    return any(excl.lower() in path.lower() for excl in EXCLUDE_DIRS)

def find_files_by_extension(root_path="C:\\", extensions=ALLOWED_EXTS):
    matched = []
    def scan_dir(path):
        try:
            for entry in os.scandir(path):
                if entry.is_dir(follow_symlinks=False):
                    if not should_exclude(entry.path):
                        scan_dir(entry.path)
                elif entry.is_file() and entry.name.lower().endswith(extensions):
                    if not (entry.name.startswith('~') or entry.name.startswith('.')):
                        matched.append(entry.path)
        except PermissionError:
            pass
        except OSError as e:
            print(f"Access error: {e}")
    scan_dir(root_path)
    return matched

start = time.time()
matches = find_files_by_extension("C:\\")
print(f"Found {len(matches)} files in {time.time() - start:.1f}s")

for out in ["C:\\file_paths.json", "file_paths.json"]:
    try:
        with open(out, "w", encoding="utf-8") as f:
            json.dump(matches, f, ensure_ascii=False, indent=2)
        print(f"Saved to: {out}")
        break
    except Exception as e:
        print(f"Could not write to {out}: {e}")


Found 2648 files in 39.8s
Could not write to C:\file_paths.json: [Errno 13] Permission denied: 'C:\\file_paths.json'
Saved to: file_paths.json


In [17]:
len(docs)

2635

In [21]:
type(docs[0])

langchain_core.documents.base.Document

In [ ]:
import json
from pathlib import Path
file_path = 'c_drive_file_index.json'
from langchain.document_loaders import JSONLoader
jq_schema = '.[] | {text: ., metadata: {path: .}}'

data = json.loads(Path(file_path).read_text())

# If the file is empty, avoid calling the loader
if not data:
    docs = []
else:
    loader = JSONLoader(
        file_path=file_path,
        jq_schema=jq_schema,
        text_content=False
    )
    docs = loader.load()


In [ ]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()
from langchain_chroma import Chroma

persist_directory = "./chroma_db"
vectorstore = Chroma(
        collection_name="paths",
        embedding_function=embeddings,
        persist_directory=persist_directory
)

In [19]:
from langchain_chroma import Chroma

persist_directory = "./chroma_db"
vectorstore = Chroma(
        collection_name="paths",
        embedding_function=embeddings,
        persist_directory=persist_directory
)

In [23]:
vectorstore.add_documents(docs)

['5f1e15ce-5ee4-4ba4-868f-a5a1ec5b4ecd',
 'bc271984-4aa7-47b5-9551-4f04440ab1bd',
 'c9f18ae8-a934-4528-8bfb-2d9e39fc02d2',
 'e9b7b604-c53b-462f-9bdc-9c17f766c868',
 'cbd3f77b-f532-4ca2-9d2e-ccad90a85345',
 '8488fe18-952b-4acb-9245-51c3ee5c3f15',
 '0603b58a-7192-4d4c-86d4-3c108eec88ba',
 'cbf42b0a-66b0-44ad-9e96-329c429c27c1',
 '3f3c71d7-96d3-42b8-997e-1530ca7eefab',
 'c1de3a3c-4b2e-4ca5-b655-a0a89964fdff',
 '5f30d39e-2420-49e8-b0fe-acdab3063403',
 '53ab748f-553b-4c8e-ac31-0e37264f3e05',
 '89565153-8d19-4901-b0fc-0b12efd6276a',
 'edfda477-6d5a-4308-9b2b-a516bef017e1',
 '4fedc865-246b-4859-b3bf-10ed361ec553',
 'a902e835-42e4-4d0a-9a6b-4b2188aeb40e',
 '493120ed-b47e-4294-99de-28990c796e16',
 '0cd4d1b8-8644-417c-9231-e8c8fcdf8c38',
 '93dfc22d-7232-4e23-a1e0-5fd263dbc8b3',
 '5466978c-722f-4572-81c3-abd68945cf6a',
 '23ae5744-2fee-41af-a480-3ba18d2198fb',
 '959792ce-7599-415d-b7cd-cd55f2d117cb',
 '228229b6-6721-4d4b-b1d6-fc1a9c2592d1',
 '6f979092-01ae-4239-9974-038e42af9c1c',
 'e38260bf-11ea-

In [25]:
vectorstore.similarity_search("caste")

[Document(id='a19480be-c99a-4279-b19e-b97c57143807', metadata={'source': 'C:\\code\\local_agent\\Local_filesearch_agent\\c_drive_file_index.json', 'seq_num': 555}, page_content='{"text": "C:\\\\KshitijDocs\\\\KshitijDocs\\\\caste cert.jpg", "metadata": {"path": "C:\\\\KshitijDocs\\\\KshitijDocs\\\\caste cert.jpg"}}'),
 Document(id='52b650c4-0685-46bb-b0fc-709efa04e760', metadata={'source': 'C:\\code\\local_agent\\Local_filesearch_agent\\c_drive_file_index.json', 'seq_num': 341}, page_content='{"text": "C:\\\\code\\\\kshitijPy\\\\documentMerger\\\\casteCer.pdf", "metadata": {"path": "C:\\\\code\\\\kshitijPy\\\\documentMerger\\\\casteCer.pdf"}}'),
 Document(id='f8b5078f-1783-4adc-be6a-98955ae1337e', metadata={'seq_num': 518, 'source': 'C:\\code\\local_agent\\Local_filesearch_agent\\c_drive_file_index.json'}, page_content='{"text": "C:\\\\KshitijDocs\\\\chotudocs\\\\castecertificate.jpg", "metadata": {"path": "C:\\\\KshitijDocs\\\\chotudocs\\\\castecertificate.jpg"}}'),
 Document(id='edc4

In [7]:
all_drives = psutil.disk_partitions()
all_drives

[sdiskpart(device='C:\\', mountpoint='C:\\', fstype='NTFS', opts='rw,fixed'),
 sdiskpart(device='D:\\', mountpoint='D:\\', fstype='FAT32', opts='rw,removable')]

In [ ]:
import psutil
import os

def analyze_drives():
    """
    Identifies and categorizes all connected drives as either 'Local' or 'Network'.
    """
    all_drives = psutil.disk_partitions()
    categorized_drives = []

    for partition in all_drives:
        drive_type = ""
        # In Windows, the 'opts' string can indicate a network drive.
        # Also, the device path for mapped network drives often starts with '\\'.
        # For other OSes, checking the file system type can be an indicator.
        if 'remote' in partition.opts or partition.device.startswith('\\\\'):
            drive_type = "Network"
        # The fstype can also give clues on Windows for non-network drives
        elif partition.fstype:
            drive_type = "Local"
        else:
            # This can happen for drives that are present but not ready (e.g., empty CD-ROM)
            drive_type = "Unknown"

        categorized_drives.append({
            "device": partition.device,
            "mountpoint": partition.mountpoint,
            "fstype": partition.fstype,
            "type": drive_type
        })
    return categorized_drives

drives = analyze_drives()
drives


[{'device': 'C:\\', 'mountpoint': 'C:\\', 'fstype': 'NTFS', 'type': 'Local'},
 {'device': 'D:\\', 'mountpoint': 'D:\\', 'fstype': 'FAT32', 'type': 'Local'}]

In [ ]:
drives[0]['device']

'C:\\'

In [ ]:
import os, json, time

ALLOWED_EXTS = ('.pdf', '.doc', '.docx', '.jpg', '.jpeg', '.png', '.ppt', '.pptx', '.xls', '.xlsx', '.csv')
EXCLUDE_DIRS = ["Windows", "Program Files", "Program Files (x86)", "PerfLogs", "$Recycle.Bin", "System Volume Information", "AppData", "Microsoft"]


def should_exclude(path):
    return any(excl.lower() in path.lower() for excl in EXCLUDE_DIRS)

def find_files_by_extension(root_path="C:\\", extensions=ALLOWED_EXTS):
    matched = []
    def scan_dir(path):
        try:
            for entry in os.scandir(path):
                if entry.is_dir(follow_symlinks=False):
                    if not should_exclude(entry.path):
                        scan_dir(entry.path)
                elif entry.is_file() and entry.name.lower().endswith(extensions):
                    if not (entry.name.startswith('~') or entry.name.startswith('.')):
                        matched.append(entry.path)
        except PermissionError:
            pass
        except OSError as e:
            print(f"Access error: {e}")
    scan_dir(root_path)
    return matched

start = time.time()
matches = find_files_by_extension("C:\\")
print(type(matches))
print(f"Found {len(matches)} files in {time.time() - start:.1f}s")

for out in ["file_paths.json"]:
    try:
        with open(out, "w", encoding="utf-8") as f:
            json.dump(matches, f, ensure_ascii=False, indent=2)
        print(f"Saved to: {out}")
        break
    except Exception as e:
        print(f"Could not write to {out}: {e}")


<class 'list'>
Found 2650 files in 32.1s
Saved to: file_paths.json


In [ ]:
# Updated pipeline cell: detect drives, gather file paths, remove existing chroma db, create/retry Chroma vectorstore
import os
import time
import shutil
import logging
import psutil
from pathlib import Path
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

logging.basicConfig(level=logging.INFO)

class Detect_and_Create_file_VStore:
    """Detect drives, find files by extension and create a Chroma vectorstore.
       Removes existing chroma_db if present and retries on failures."""
    ALLOWED_EXTS = ('.pdf', '.doc', '.docx', '.jpg', '.jpeg', '.png', '.ppt', '.pptx', '.xls', '.xlsx', '.csv')
    EXCLUDE_DIRS = ["Windows", "Program Files", "Program Files (x86)", "PerfLogs", "$Recycle.Bin",
                    "System Volume Information", "AppData", "Microsoft"]

    def __init__(self, persist_directory="./chroma_db"):
        self.persist_directory = persist_directory
        self.vectorstore = None

    def should_exclude(self, path):
        return any(excl.lower() in path.lower() for excl in self.EXCLUDE_DIRS)

    def find_files_by_extension(self, root_path, extensions=None):
        extensions = extensions or self.ALLOWED_EXTS
        matched = []
        def scan_dir(path):
            try:
                for entry in os.scandir(path):
                    if entry.is_dir(follow_symlinks=False):
                        if not self.should_exclude(entry.path):
                            scan_dir(entry.path)
                    elif entry.is_file() and entry.name.lower().endswith(extensions):
                        if not (entry.name.startswith('~') or entry.name.startswith('.')):
                            matched.append(entry.path)
            except (PermissionError, OSError):
                # skip directories we can't access
                return
        scan_dir(root_path)
        return matched

    def _remove_existing_db(self):
        if os.path.exists(self.persist_directory):
            try:
                logging.info("Removing existing Chroma DB at %s", self.persist_directory)
                shutil.rmtree(self.persist_directory)
            except Exception as e:
                logging.warning("Could not remove existing chroma db: %s", e)

    def create_file_vectorstore(self, file_paths, retries=3, delay=2):
        # create Document objects for Chroma
        docs = [Document(page_content=fp, metadata={"path": fp}) for fp in file_paths]

        # remove existing DB before creating a fresh one
        self._remove_existing_db()

        embeddings = OpenAIEmbeddings()

        last_exc = None
        for attempt in range(1, retries + 1):
            try:
                vs = Chroma(collection_name="paths", embedding_function=embeddings, persist_directory=self.persist_directory)
                # add documents (this will create DB files)
                vs.add_documents(docs)
                # persist if available
                if hasattr(vs, "persist"):
                    try:
                        vs.persist()
                    except Exception:
                        pass
                self.vectorstore = vs
                logging.info("Chroma vectorstore created successfully on attempt %d", attempt)
                return vs
            except Exception as e:
                last_exc = e
                logging.exception("Attempt %d: Failed to create Chroma vectorstore: %s", attempt, e)
                if attempt < retries:
                    time.sleep(delay)
        # if all attempts failed, raise the last exception
        raise RuntimeError("Failed to create Chroma vectorstore") from last_exc

    def run_pipeline(self, root_dirs=None):
        # default to all mounted partitions' mountpoints
        if root_dirs is None:
            root_dirs = [p.mountpoint for p in psutil.disk_partitions()]
        all_paths = []
        for root in root_dirs:
            try:
                logging.info("Scanning drive: %s", root)
                all_paths.extend(self.find_files_by_extension(root))
            except Exception as e:
                logging.warning("Skipping drive %s due to error: %s", root, e)
        logging.info("Found %d matching files", len(all_paths))
        return self.create_file_vectorstore(all_paths)

# Example usage (will run when this cell is executed):
try:
    obj = Detect_and_Create_file_VStore()
    vectorstore = obj.run_pipeline()
    logging.info("Pipeline completed, vectorstore: %s", type(vectorstore))
except Exception as e:
    logging.exception("Pipeline failed: %s", e)

INFO:root:Scanning drive: C:\
INFO:root:Found 3381 matching files
INFO:root:Removing existing Chroma DB at ./chroma_db
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:root:Chroma vectorstore created successfully on attempt 1
INFO:root:Pipeline completed, vectorstore: <class 'langchain_chroma.vectorstores.Chroma'>


In [40]:
vectorstore.similarity_search("project report")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(id='d1fc87f4-a131-42b8-a43a-d96562a5c1b2', metadata={'path': 'C:\\Users\\hp\\Downloads\\minor project report final (3).pdf'}, page_content='C:\\Users\\hp\\Downloads\\minor project report final (3).pdf'),
 Document(id='fe2ac6fe-8b2b-4ffc-9fa9-ba237ab91e0f', metadata={'path': 'C:\\Users\\hp\\Downloads\\minor project report final.pdf'}, page_content='C:\\Users\\hp\\Downloads\\minor project report final.pdf'),
 Document(id='9a984015-4a5c-4e4e-a663-76e3bb19f3f7', metadata={'path': 'C:\\Users\\hp\\Downloads\\minor project report final (1).pdf'}, page_content='C:\\Users\\hp\\Downloads\\minor project report final (1).pdf'),
 Document(id='62deefaf-3614-464f-b79d-14fef3984933', metadata={'path': 'C:\\Users\\hp\\Downloads\\minor project report final (2).pdf'}, page_content='C:\\Users\\hp\\Downloads\\minor project report final (2).pdf')]

In [26]:
obj = Detect_and_Create_file_VStore()
vectorstore = obj.run_pipeline()

In [38]:
results = vectorstore.similarity_search("project report", k = 10)
clean_results = [res.page_content.split(", \"metadata\"")[0] for res in results]
clean_results

['{"text": "C:\\\\code\\\\UdemyGenai\\\\Langchain\\\\ProcodebaseAssignement\\\\Report.pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\VCET UG Mini Project Report Model 12-10-19.docx"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\MINOR_PROJECT_REPORT_FORMAT_JIT.pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Desktop\\\\clg\\\\Major project final report.pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\minor project report final (3).pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\minor project report final (2).pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\minor project report final (1).pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\minor project report final.pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Desktop\\\\clg\\\\MINOR_PROJECT_REPORT_FORMAT_JIT.pdf"',
 '{"text": "C:\\\\Users\\\\hp\\\\Downloads\\\\MINOR_PROJECT_REPORT_FORMAT_JIT.docx"']

In [22]:
list = psutil.disk_partitions()


In [23]:
list[0].mountpoint

'C:\\'